### Learning Path Task 10 : Face Recognition Using CNN

In [ ]:
#install libraries
!pip install pandas
!pip install pandas-datareader
!pip install matplotlib
!pip install pillow
!pip install requests
!pip install h5py
!pip install tensorflow==1.8.0
!pip install keras==2.2.0
!pip install opencv-python
!pip install opencv-contrib-python
!pip install keras-preprocessing==1.1.2
!pip install adam



ERROR: Could not find a version that satisfies the requirement tensorflow==1.8.0 (from versions: 1.13.1, 1.13.2, 1.14.0, 1.15.0, 1.15.2, 1.15.3, 1.15.4, 1.15.5, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.2.0rc0, 2.2.0rc1, 2.2.0rc2, 2.2.0rc3, 2.2.0rc4, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0rc0, 2.3.0rc1, 2.3.0rc2, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0rc0, 2.4.0rc1, 2.4.0rc2, 2.4.0rc3, 2.4.0rc4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0rc0, 2.5.0rc1, 2.5.0rc2, 2.5.0rc3, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.8.0rc0, 2.8.0rc1, 2.8.0)
ERROR: No matching distribution found for tensorflow==1.8.0
     |████████████████████████████████| 300 kB 5.2 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
  Attempting uninstall: keras-preprocessing
    Found existing installation: Keras-Preprocessing 1.1.2
    Uninstalling Keras-Preprocessing-1.1.2:
      Successfully uninstalled Ker

In [1]:
# Importing necesary libraries 
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import TensorBoard

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from keras.utils import np_utils
import itertools

In [3]:
#Loading the Dataset
data = np.load('faces.npz')
print(data.files)

['testY', 'testX', 'trainX', 'trainY']


In [4]:
# Load Training Images
X_train=data['trainX']
print("Orignal shape : ",X_train.shape)
print("Orignal Training Data : ",X_train)


Orignal shape :  (240, 10304)
Orignal Training Data :  [[ 48.  49.  45. ...  47.  46.  46.]
 [ 60.  60.  62. ...  32.  34.  34.]
 [ 39.  44.  53. ...  29.  26.  29.]
 ...
 [114. 117. 114. ...  98.  96.  98.]
 [105. 105. 107. ...  54.  47.  41.]
 [116. 114. 117. ...  95. 100. 101.]]


In [5]:
# Loading Testing Images
X_test = data['testX']
print("Orignal shape : ",X_test.shape)
print("Orignal Testing Data : ",X_test)

Orignal shape :  (160, 10304)
Orignal Testing Data :  [[ 41.  47.  47. ...  35.  37.  38.]
 [ 44.  43.  32. ...  43.  43.  37.]
 [ 42.  41.  44. ...  42.  43.  41.]
 ...
 [101. 100. 103. ...  31.  40.  42.]
 [105. 108. 106. ...  44.  40.  47.]
 [113. 114. 111. ...  62.  81.  89.]]


In [6]:
#Note :After loading the Dataset you have to normalize every image.
#And an image is a Uint8 matrix of pixels and for calculation, you need to 
#convert the format of the image to float or double


In [7]:
#Converting type to float
print("Type of X_train, X_test",X_train.dtype,X_train.dtype)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
print("Type of X_train, X_test",X_train.dtype,X_train.dtype)

Type of X_train, X_test float64 float64
Type of X_train, X_test float32 float32


In [8]:
#Converting all values of X-train and test between 0-1 by dividing it by 255 (for the color)
print("Previous activations : ",X_train[X_train>0])
X_train /= 255
X_test /= 255
print("New activations : ",X_train[X_train>0])

Previous activations :  [ 48.  49.  45. ...  95. 100. 101.]
New activations :  [0.1882353  0.19215687 0.1764706  ... 0.37254903 0.39215687 0.39607844]


In [9]:
#Labels for our set
Y_train= data['trainY']
Y_test= data['testY']
Y_test

array([ 0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  2,
        2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,
        4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,
        6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,
        8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10,
       10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12,
       12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14,
       14, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16,
       17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 19,
       19, 19, 19, 19, 19, 19, 19], dtype=uint8)

In [10]:
#Split DataSet : Validation data and Train
#Why do this  : Validation DataSet: this data set is used to minimize 
#              overfitting.If the accuracy over the training data set increases,  
#              but the accuracy over then validation data set stays the same or 
#              decreases, then you're overfitting your neural network and you 
#              should stop training.

note=""" Note: We usually use 30 percent of every dataset as the validation data
         but Here we only used 5 percent because the number of images in this 
          dataset is very low."""


In [11]:
#Train test split 
x_train, x_valid, y_train, y_valid= train_test_split(
    X_train, Y_train, test_size=.05, random_state=123)

In [12]:
#Reshaping our Images
im_rows=112
im_cols=92
batch_size=512
im_shape=(im_rows, im_cols, 1)

#change the size of images
x_train = x_train.reshape(x_train.shape[0], *im_shape)
x_test = X_test.reshape(X_test.shape[0], *im_shape)
x_valid = x_valid.reshape(x_valid.shape[0], *im_shape)

print('y_train shape: {}'.format(y_train.shape[0]))
print('Y_test shape: {}'.format(Y_test.shape))

x_train shape: 228
x_test shape: (160,)


In [16]:
note_1=""" Build CNN model: CNN have 3 main layer:

1-Convolotional layer
2- pooling layer
3- fully connected layer

we could build a new architecture of CNN by changing the number and position of layers."""
#filters= the depth of output image or kernels

cnn_model= Sequential([
    Conv2D(filters=36, kernel_size=7, activation='relu', input_shape= im_shape),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=54, kernel_size=5, activation='relu', input_shape= im_shape),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(2024, activation='relu'),
     Dropout(0.5),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    #20 is the number of outputs
    Dense(20, activation='softmax')  
])

cnn_model.compile(
    loss='sparse_categorical_crossentropy',#'categorical_crossentropy',
    optimizer=Adam(lr=0.0001),
    metrics=['accuracy']
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [17]:
cnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 106, 86, 36)       1800      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 53, 43, 36)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 49, 39, 54)        48654     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 24, 19, 54)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 24624)             0         
                                                                 
 dense_2 (Dense)             (None, 2024)             

In [18]:
note_2=""" Train the Model , change the number of epochs for variations"""
history=cnn_model.fit(
    np.array(x_train), np.array(y_train), batch_size=512,
    epochs=250, verbose=2,
    validation_data=(np.array(x_valid),np.array(y_valid)),
)

Epoch 1/250
1/1 - 12s - loss: 2.9973 - accuracy: 0.0570 - val_loss: 3.0003 - val_accuracy: 0.0000e+00 - 12s/epoch - 12s/step
Epoch 2/250
1/1 - 0s - loss: 2.9962 - accuracy: 0.0351 - val_loss: 3.0046 - val_accuracy: 0.0000e+00 - 210ms/epoch - 210ms/step
Epoch 3/250
1/1 - 0s - loss: 3.0120 - accuracy: 0.0658 - val_loss: 3.0136 - val_accuracy: 0.0000e+00 - 200ms/epoch - 200ms/step
Epoch 4/250
1/1 - 0s - loss: 2.9772 - accuracy: 0.0614 - val_loss: 3.0100 - val_accuracy: 0.0000e+00 - 210ms/epoch - 210ms/step
Epoch 5/250
1/1 - 0s - loss: 2.9842 - accuracy: 0.0746 - val_loss: 3.0066 - val_accuracy: 0.0000e+00 - 224ms/epoch - 224ms/step
Epoch 6/250
1/1 - 0s - loss: 2.9958 - accuracy: 0.0833 - val_loss: 3.0053 - val_accuracy: 0.0000e+00 - 205ms/epoch - 205ms/step
Epoch 7/250
1/1 - 0s - loss: 2.9854 - accuracy: 0.0526 - val_loss: 3.0028 - val_accuracy: 0.0000e+00 - 203ms/epoch - 203ms/step
Epoch 8/250
1/1 - 0s - loss: 2.9968 - accuracy: 0.0658 - val_loss: 3.0003 - val_accuracy: 0.0000e+00 - 207m

In [23]:
note_3=""" Evaluating the test data"""
scor = cnn_model.evaluate( np.array(x_test),  np.array(Y_test), verbose=0)

print('test los {:.4f} %'.format(scor[0]*100))
print('test acc {:.4f} %'.format(scor[1]*100))

test los 38.1786 %
test acc 91.2500 %
